In [1]:
import pandas as pd
from tensorflow import keras

import numpy as np

from tqdm import tqdm

import cv2 as cv

from cv2 import imread
from cv2 import IMREAD_COLOR
from skimage.transform import resize

In [2]:
# Neural/conv & Co networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pickle
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from string import punctuation
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7704376036267331634
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4963368960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5461297678523710374
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            #tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
    except RuntimeError as e:
        print(e)

In [7]:
df = pd.read_csv("train_clean.csv")
df["color_tags"] = pd.read_csv("y_train.csv")["color_tags"]
df.head()

,item_description,image_file_name,color_tags
0,sankyo aluminum shade beam standing type one s...,278003_10389968_1.jpg,"['Silver', 'Grey', 'Black']"
1,sale sale fashion coordination thick sash belt...,220810_10010506_1.jpg,"['Brown', 'Black']"
2,geta paulownia made japan woman tone nose widt...,207456_10045549_1.jpg,"['White', 'Black']"
3,limited time yen coupon issuance shoe box widt...,346541_10000214_1.jpg,"['Beige', 'Brown', 'Black']"
4,post mailbox mailbox post multi family housing...,240426_10024071_1.jpg,['Silver']


In [8]:
df[df.isna().any(axis=1)]

,item_description,image_file_name,color_tags
4172,NaN,284861_10152178_1.gif,['Blue']
196161,NaN,284861_10152142_1.gif,['Blue']


In [9]:
df["color_tags"] = df["color_tags"].apply(lambda x: x[1:-1].replace("'","").split(','))
unique_colors = set()
for line in df["color_tags"]:
    for color in line:
        unique_colors.add(color)
        
df["image_file_name"] = df["image_file_name"].apply(lambda x: "./images/"+x)

In [10]:
df.head()

,item_description,image_file_name,color_tags
0,sankyo aluminum shade beam standing type one s...,./images/278003_10389968_1.jpg,"[Silver, Grey, Black]"
1,sale sale fashion coordination thick sash belt...,./images/220810_10010506_1.jpg,"[Brown, Black]"
2,geta paulownia made japan woman tone nose widt...,./images/207456_10045549_1.jpg,"[White, Black]"
3,limited time yen coupon issuance shoe box widt...,./images/346541_10000214_1.jpg,"[Beige, Brown, Black]"
4,post mailbox mailbox post multi family housing...,./images/240426_10024071_1.jpg,[Silver]


In [11]:
df.dropna(inplace=True)
MAX_SEQUENCE_LENGTH = df['item_description'].map(len).max()
MAX_SEQUENCE_LENGTH

9008

In [12]:
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(df['item_description'])
word_index = tokenizer.word_index
df['tokenized_items'] = tokenizer.texts_to_sequences(df['item_description'])

In [13]:
df.head()

,item_description,image_file_name,color_tags,tokenized_items
0,sankyo aluminum shade beam standing type one s...,./images/278003_10389968_1.jpg,"[Silver, Grey, Black]","[4396, 619, 962, 2506, 2388, 17, 31, 772, 382,..."
1,sale sale fashion coordination thick sash belt...,./images/220810_10010506_1.jpg,"[Brown, Black]","[162, 162, 207, 775, 530, 1482, 377, 1482, 377..."
2,geta paulownia made japan woman tone nose widt...,./images/207456_10045549_1.jpg,"[White, Black]","[4459, 2282, 24, 88, 50, 878, 1885, 10, 211, 1..."
3,limited time yen coupon issuance shoe box widt...,./images/346541_10000214_1.jpg,"[Beige, Brown, Black]","[453, 16, 72, 779, 9629, 105, 166, 10, 140, 61..."
4,post mailbox mailbox post multi family housing...,./images/240426_10024071_1.jpg,[Silver],"[698, 3116, 3116, 698, 557, 799, 3487, 7328, 2..."


In [14]:
sequence = pad_sequences(df["tokenized_items"])  
df_tmp=pd.DataFrame(sequence)
s = df_tmp.apply(lambda row: row.to_list(),axis=1)
df["sequences"] = s

In [15]:
MAX_SEQUENCE = len(sequence[0])
MAX_SEQUENCE

1399

In [16]:
df.head()

,item_description,image_file_name,color_tags,tokenized_items,sequences
0,sankyo aluminum shade beam standing type one s...,./images/278003_10389968_1.jpg,"[Silver, Grey, Black]","[4396, 619, 962, 2506, 2388, 17, 31, 772, 382,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,sale sale fashion coordination thick sash belt...,./images/220810_10010506_1.jpg,"[Brown, Black]","[162, 162, 207, 775, 530, 1482, 377, 1482, 377...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,geta paulownia made japan woman tone nose widt...,./images/207456_10045549_1.jpg,"[White, Black]","[4459, 2282, 24, 88, 50, 878, 1885, 10, 211, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,limited time yen coupon issuance shoe box widt...,./images/346541_10000214_1.jpg,"[Beige, Brown, Black]","[453, 16, 72, 779, 9629, 105, 166, 10, 140, 61...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,post mailbox mailbox post multi family housing...,./images/240426_10024071_1.jpg,[Silver],"[698, 3116, 3116, 698, 557, 799, 3487, 7328, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[["image_file_name","sequences"]],df["color_tags"], test_size=0.2, random_state=1)
print(X_train.shape)
print(X_test.shape)
print()
print(y_train.shape)
print(y_test.shape)

(169694, 2)
(42424, 2)

(169694,)
(42424,)


In [18]:
X_train.head()

,image_file_name,sequences
85035,./images/296970_10506152_1.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
93012,./images/252082_10067475_1.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
122258,./images/295152_21491524_1.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
128746,./images/210806_10022238_1.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
155869,./images/354562_10086498_1.jpg,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
from sklearn.preprocessing import LabelBinarizer

class Custom_Generator(keras.utils.Sequence) :
    
    def __init__(self, data, labels, batch_size, size_image=-1):
        self.image = data.iloc[:,0]
        self.text = data.iloc[:,1]
        self.labels = labels
        self.batch_size = batch_size
        self.size = size_image
        self.encoder = LabelBinarizer().fit(list(unique_colors))
        
    def __len__(self):
        return (np.ceil(len(self.labels) / float(self.batch_size))).astype(np.int)
    
    def open_gif(self,image):
        gif = cv.VideoCapture(image)
        ret, frame = gif.read()
        cv.destroyAllWindows()
        return frame
    
    def __getitem__(self, idx):
        batch_image = self.image[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_text = self.text[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        
        images = []
        labels = []
        
        for image in batch_image:
            if not size==-1 :
                if ".gif" in image : image = resize(self.open_gif(image),size)
                else : image = resize(imread(image,IMREAD_COLOR),size)
            else : 
                if ".gif" in image : image = self.open_gif(image)
                else : image = imread(image,IMREAD_COLOR)

            image = image/255.0
            images.append(image)
            
        for y in batch_y:
            labels.append( np.sum( self.encoder.transform(y), axis=0 ) )
            
        images = np.asarray(images)
        texts = np.asarray(batch_text)
        labels = np.asarray(labels)
        
        return [images,texts], labels, [None]
    
batch_size = 2
size = (75, 115, 3)

gen_train = Custom_Generator(X_train, y_train, batch_size, size)
gen_test = Custom_Generator(X_test, y_test, batch_size, size)

In [20]:
%%capture
def get_embedding_matrix(typeToLoad):
    if typeToLoad == "glove":
        EMBEDDING_FILE="/content/glove.twitter.27B.100d.txt"
        embed_size = 100
    elif typeToLoad == "word2vec":
        word2vecDict = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
        embed_size = 300
    elif typeToLoad == "fasttext":
        EMBEDDING_FILE="/content/wiki-news-300d-1M.vec"
        embed_size = 300

    if typeToLoad == "glove" or typeToLoad == "fasttext":
        embeddings_index = dict()
        f = open(EMBEDDING_FILE)
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()
        print("Loaded " + str(len(embeddings_index)) + " word vectors.")
    else:
        embeddings_index = dict()
        for word in word2vecDict.wv.vocab:
            embeddings_index[word] = word2vecDict.word_vec(word)
        print("Loaded " + str(len(embeddings_index)) + " word vectors.")

        embedding_matrix = 1 * np.random.randn(len(word_index)+1, embed_size)

        embeddedCount = 0
    for word, i in word_index.items():
        i-=1
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
            embeddedCount+=1
        print("total embedded:", embeddedCount, "common words")

    del(embeddings_index)

    return embedding_matrix

word2vec_embedding_matrix = get_embedding_matrix("word2vec")

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Input, Bidirectional, Flatten, Conv2D, MaxPooling2D, concatenate, Conv1D, MaxPooling1D
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

In [22]:
def compile_model(embedding_matrix):
    lstm_input = Input(shape=(batch_size,))
    x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], mask_zero=True, input_length=MAX_SEQUENCE_LENGTH, trainable=False)(lstm_input)
    x = Dropout(0.3)(x)
    x = LSTM(64, return_sequences = True)(x)
    x = Dropout(0.3)(x)
    x = LSTM(64)(x)
    x = Dropout(0.3)(x)
    lstm_out = Dense(18, activation = 'relu')(x)

    print(size[0], size[1], size[2])
    cnn_input = Input(shape=(size[0], size[1], size[2]))
    y = Conv2D(32, (3, 3), activation='relu', input_shape=(size[0], size[1], size[2]))(cnn_input)
    y = MaxPooling2D(2, 2)(y)
    y = Conv2D(64, (3, 3), activation='relu')(y)
    y = MaxPooling2D(2, 2)(y)
    y = Conv2D(128, (3, 3), activation='relu')(y)
    y = MaxPooling2D(2, 2)(y)
    y = Conv2D(128, (3, 3), activation='relu')(y)
    y = MaxPooling2D(2, 2)(y)
    y = Flatten()(y)
    y = Dropout(0.3)(y)
    cnn_out = Dense(512, activation='relu')(y)

    concat_inp = concatenate([cnn_out, lstm_out])
    z = Dense(256, activation='relu')(concat_inp)
    z = Dropout(0.3)(z)
    z = Dense(128, activation='relu')(z)
    z = Dropout(0.3)(z)
    output = Dense(len(list(unique_colors)), activation='sigmoid')(z)

    model = Model(inputs=[cnn_input, lstm_input], outputs=[output])
    adam = Adam(lr=0.001, decay=1e-5)

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [23]:
word2vec_model = compile_model(word2vec_embedding_matrix)
word2vec_model.summary()

75 115 3
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 75, 115, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 73, 113, 32)  896         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 36, 56, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 34, 54, 64)   18496       max_pooling2d[0][0]              
_____________________________________________________________________________________

In [24]:
history = word2vec_model.fit_generator(
    generator = gen_train,
    epochs=20,
    validation_data=gen_test)

Instructions for updating:
Please use Model.fit, which supports generators.


ValueError: in converted code:

    E:\Anaconda\envs\tf2\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py:677 map_fn
        batch_size=None)
    E:\Anaconda\envs\tf2\lib\site-packages\tensorflow_core\python\keras\engine\training.py:2410 _standardize_tensors
        exception_prefix='input')
    E:\Anaconda\envs\tf2\lib\site-packages\tensorflow_core\python\keras\engine\training_utils.py:582 standardize_input_data
        str(data_shape))

    ValueError: Error when checking input: expected input_1 to have shape (2,) but got array with shape (1,)
